In [27]:
using Pkg;
Pkg.activate("..");
Pkg.instantiate();

  Activating project at `~/dev/uni/amo-individual-project`


In [36]:
import CSV

In [28]:
include.(filter(contains(r".jl$"), readdir("../src"; join=true)))
println()

In [29]:
ieee_files = filter(contains("ieee"), readdir("../data"; join=true))

5-element Vector{String}:
 "../data/ieee118cdf.txt"
 "../data/ieee14cdf.txt"
 "../data/ieee300cdf.txt"
 "../data/ieee30cdf.txt"
 "../data/ieee57cdf.txt"

In [33]:
import Dates
using DataFrames

function benchmark(files::Vector{String})::DataFrame

	results_df = DataFrame(
		:file => String[],
		:method => String[],
		:elapsed => Float64[],
		:objective => Float64[],
		:has_converged => Bool[],
	)

	for file in files
		busses, lines = read_IEEE_common_data_format(file)
		for mode in ["ADMM-single-threaded", "ADMM-multi-threaded", "simple-lp"]
			println("Solving $file with $mode")
			start = Dates.now()
			objective = Inf
			elapsed = Inf
			has_converged = false

			if mode == "simple-lp"
				_, _, objective = solve_with_lp(busses, lines)
				has_converged = true
			else 
				config = create_admm_config(
						Dict(
							:ρ => 1.0, 
							:use_multithreading => mode == "ADMM-multi-threaded",
							:silent => true,
						)
					)
				_, _, objective, has_converged = solve_with_admm(busses, lines, config)
			end

			elapsed = (Dates.now() - start).value / 1000
			push!(results_df, (file, mode, elapsed, objective, has_converged))
		end
	end

	return results_df
end

benchmark (generic function with 1 method)

In [34]:
results = benchmark(ieee_files)

Solving ../data/ieee118cdf.txt with ADMM-single-threaded
Solving ../data/ieee118cdf.txt with ADMM-multi-threaded
Solving ../data/ieee118cdf.txt with simple-lp
Solving ../data/ieee14cdf.txt with ADMM-single-threaded
Solving ../data/ieee14cdf.txt with ADMM-multi-threaded
Solving ../data/ieee14cdf.txt with simple-lp
Solving ../data/ieee300cdf.txt with ADMM-single-threaded
Solving ../data/ieee300cdf.txt with ADMM-multi-threaded
Solving ../data/ieee300cdf.txt with simple-lp
Solving ../data/ieee30cdf.txt with ADMM-single-threaded
Solving ../data/ieee30cdf.txt with ADMM-multi-threaded
Solving ../data/ieee30cdf.txt with simple-lp
Solving ../data/ieee57cdf.txt with ADMM-single-threaded
Solving ../data/ieee57cdf.txt with ADMM-multi-threaded
Solving ../data/ieee57cdf.txt with simple-lp


Row,file,method,elapsed,objective,has_converged
,String,String,Float64,Float64,Bool
1,../data/ieee118cdf.txt,ADMM-single-threaded,52.722,128380.0,true
2,../data/ieee118cdf.txt,ADMM-multi-threaded,9.924,128380.0,true
3,../data/ieee118cdf.txt,simple-lp,0.003,128380.0,true
4,../data/ieee14cdf.txt,ADMM-single-threaded,5.58,9065.08,true
5,../data/ieee14cdf.txt,ADMM-multi-threaded,1.103,9065.08,true
6,../data/ieee14cdf.txt,simple-lp,0.001,9065.0,true
7,../data/ieee300cdf.txt,ADMM-single-threaded,276.015,8.36183e5,false
8,../data/ieee300cdf.txt,ADMM-multi-threaded,63.993,8.36183e5,false
9,../data/ieee300cdf.txt,simple-lp,0.005,8.36166e5,true


In [38]:
CSV.write("../results/results.csv", results)

"../results/results.csv"